In [9]:
import requests
from bs4 import BeautifulSoup
import json
import chompjs
import re
import pandas as pd
import os

In [10]:
rx_obj = re.compile(r'(\[\{.+\}\])')

In [11]:
def extract_data(url):
        response = requests.get(url)

        soup = BeautifulSoup(response.text)
        
        results = soup.find('div' , string=re.compile("土地面積"))
        land_area = -1
        for r in results:
                land_area = r.parent.findNext('td',{'class':'w290 bdCell'}).getText().strip()
        

        #print (land_area)

        data = soup.find('script', type='text/javascript').text.replace('\n', ' ').replace('\t', ' ')

        obj = chompjs.parse_js_object(re.findall(rx_obj, data)[0])

        obj[0]["land_area"] = land_area
        return obj


#extract_data("https://suumo.jp/ikkodate/tokyo/sc_suginami/nc_73899376/")

In [12]:
done = set()

if os.path.exists("details.tsv"):
    for line in open('details.tsv', 'r'):
        url = line.strip().split("\t")[0]
        done.add(url)

print (done)

with open('details.tsv', 'a') as f:
    for line in open('【SUUMO】東京都中野区、東京都杉並区、東京都練馬区の新築一戸建て・分譲住宅・一軒家購入情報.csv', 'r'):
        url = line.strip()

        if url not in done:
            try:
                obj = extract_data(url)
                #print (url)
                f.write(f"{url}\t{json.dumps(obj[0], ensure_ascii=False)}\n")
            except Exception as e:
                print (f"{e}: {url}")

#print (re.findall(rx_obj, data)[0])
#print(data) 

{'https://suumo.jp/ikkodate/tokyo/sc_nakano/nc_75417537/', 'https://suumo.jp/ikkodate/tokyo/sc_nerima/nc_75240138/', 'https://suumo.jp/ikkodate/tokyo/sc_toshima/nc_75404184/', 'https://suumo.jp/ikkodate/tokyo/sc_adachi/nc_74753156/', 'https://suumo.jp/ikkodate/tokyo/sc_arakawa/nc_75189916/', 'https://suumo.jp/ikkodate/tokyo/sc_setagaya/nc_75000456/', 'https://suumo.jp/ikkodate/tokyo/sc_bunkyo/nc_74927241/', 'https://suumo.jp/ikkodate/tokyo/sc_nerima/nc_74348061/', 'https://suumo.jp/ikkodate/tokyo/sc_kita/nc_75155092/', 'https://suumo.jp/ikkodate/tokyo/sc_adachi/nc_75433849/', 'https://suumo.jp/ikkodate/tokyo/sc_edogawa/nc_75416384/', 'https://suumo.jp/ikkodate/tokyo/sc_katsushika/nc_74435634/', 'https://suumo.jp/ikkodate/tokyo/sc_nerima/nc_73788947/', 'https://suumo.jp/ikkodate/tokyo/sc_edogawa/nc_75166367/', 'https://suumo.jp/ikkodate/tokyo/sc_suginami/nc_74591495/', 'https://suumo.jp/ikkodate/tokyo/sc_edogawa/nc_75338476/', 'https://suumo.jp/ikkodate/tokyo/sc_itabashi/nc_74879666/', 

In [13]:
#obj[0]["ryoikiShuNm"]

In [14]:
content = "\t".join(["url", "bukken_title", "madori", "construction_area", "land_area", "price", "construction_year", "construction_month", "features", "todofukenNm", "shikugunNm", "tohoJikan1"]) + "\n"

#n = 0

for line in open('details.tsv', 'r'):

    fields = line.strip().split("\t")
    url = fields[0]
    data = json.loads(fields[1])

    #print (json.dumps(data, ensure_ascii=False, indent =2))

    bukken_type = data["ryoikiShuNm"]
    
    if bukken_type == "新築一戸建て":
        

        bukkenCd = data["bukkenCd"]

        features = data["tokuchoPickupList"]

        construction_year = data["chikunengetsu"][:4]
        construction_month = data["chikunengetsu"][4:]

        todofukenNm = data["todofukenNm"]
        shikugunNm = data["shikugunNm"]
        tohoJikan1 = data["tohoJikan1"]

        hanbai_kosuu = data["hanbai_kosuu"]
        
        #print (url)

        # prices = data["kakakuDisp"]
        # madoriDisp = data["madoriDisp"]
        # tatemonoMensekiDisp = data["tatemonoMensekiDisp"]

        if hanbai_kosuu == "1":
            price = data["kakakuDisp"][0]

            if price and float(price) != -1:
            #print (url)
                madori = data["madoriDisp"][0]
                if madori == "ワンルーム":
                    madori = "1DK"

                construction_area = data["tatemonoMensekiDisp"][0]
                if "m2" in data["land_area"]:
                    land_areas = re.findall(r'([0-9.]+)m2', data["land_area"])
                    if len(land_areas) > 0:
                        land_area = land_areas[0]
                        content += f"{url}\t{bukkenCd}\t{madori}\t{construction_area}\t{land_area}\t{price}\t{construction_year}\t{construction_month}\t{'|'.join(features)}\t{todofukenNm}\t{shikugunNm}\t{tohoJikan1}\n"


        else:
            for unit in data["madoriList"]:
                
                temp_price = unit["kakakuDisp"]

                if temp_price != "":
                    madori_list_empty = False
                    if float(temp_price) != -1:
                        madori = unit["madoriDisp"]
                        if madori == "ワンルーム":
                            madori = "1DK"
                        land_area = unit["tochiMensekiDisp"]
                        construction_area = unit["tatemonoMensekiDisp"]
                        title = unit["title"]

                        temp_bukkenCd = bukkenCd + '_' + title
                    
                        content += f"{url}\t{temp_bukkenCd}\t{madori}\t{construction_area}\t{land_area}\t{temp_price}\t{construction_year}\t{construction_month}\t{'|'.join(features)}\t{todofukenNm}\t{shikugunNm}\t{tohoJikan1}\n"
            
            # if madori_list_empty:
            #     content += f"{url}\t{bukkenCd}\t{madori}\t{construction_area}\t{land_area}\t{price}\t{construction_year}\t{construction_month}\t{'|'.join(features)}\t{todofukenNm}\t{shikugunNm}\t{tohoJikan1}\n"
                
                #print (url, data)
                #n += 1
                #print (json.dumps(data, ensure_ascii=False, indent =2))

    # property_type = data["ryoikiShuNm"]

    # if property_type == "中古一戸建て":
    #     print (property_type)
#print (content.strip())

with open('formatted.tsv', 'w') as f:
    f.write(content.strip())

In [15]:
df = pd.read_csv("formatted.tsv", sep="\t")
df.head()

,url,bukken_title,madori,construction_area,land_area,price,construction_year,construction_month,features,todofukenNm,shikugunNm,tohoJikan1
0,https://suumo.jp/ikkodate/tokyo/sc_shinjuku/nc...,74725415,1LDK+2S,90.05,104.62,84100000,2024.0,6.0,BELS/省エネ基準適合認定書あり|フラット３５・S適合証明書|エコポイント対象住宅|省エネ...,東京都,新宿区,6.0
1,https://suumo.jp/ikkodate/tokyo/sc_suginami/nc...,73899376_3号棟,2LDK+S,100.42,120.20,111300000,2024.0,2.0,省エネルギー対策|地盤調査済|２沿線以上利用可|ＬＤＫ２０畳以上|年度内引渡可|省エネ給湯器...,東京都,杉並区,16.0
2,https://suumo.jp/ikkodate/tokyo/sc_suginami/nc...,73899376_1号棟,4LDK,116.95,120.19,134800000,2024.0,2.0,省エネルギー対策|地盤調査済|２沿線以上利用可|ＬＤＫ２０畳以上|年度内引渡可|省エネ給湯器...,東京都,杉並区,16.0
3,https://suumo.jp/ikkodate/tokyo/sc_setagaya/nc...,74098813,3LDK,131.00,126.16,199800000,2024.0,4.0,長期優良住宅認定通知書|設計住宅性能評価書|建設住宅性能評価書（新築時）|省エネルギー対策|...,東京都,世田谷区,16.0
4,https://suumo.jp/ikkodate/tokyo/sc_adachi/nc_7...,73639918_1号棟,3LDK+S,116.14,70.26,51800000,2024.0,1.0,BELS/省エネ基準適合認定書あり|フラット３５・S適合証明書|省エネルギー対策|フラット３...,東京都,足立区,14.0


In [16]:
df[~df["madori"].str.contains("DK")]

,url,bukken_title,madori,construction_area,land_area,price,construction_year,construction_month,features,todofukenNm,shikugunNm,tohoJikan1
2438,https://suumo.jp/ikkodate/tokyo/sc_koto/nc_727...,72783972,2K,92.54,67.48,198000000,2023.0,6.0,２沿線以上利用可|スーパー 徒歩10分以内|２階建|都市ガス|小学校 徒歩10分以内,東京都,江東区,3.0
